In [501]:
import pandas as pd
import numpy as np
import sklearn.datasets as datasets
import random
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.svm import SVC, SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, mean_squared_error


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Data loading

In [502]:
SEED = 42

iris = datasets.load_iris()
diabetes = datasets.load_diabetes()
digits = datasets.load_digits()
wine = datasets.load_wine()
cancer = datasets.load_breast_cancer()

In [503]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(SEED) # Seed 고정

In [504]:
def seperate_x_y(df):
    train_y = df['target']
    train_x = df.drop('target', axis=1)
    return train_x, train_y
def make_df(data):
    df = pd.DataFrame(data.data, columns = data.feature_names)
    df['target'] = data.target
    return df
def return_shape(df): # return dataframe shape dim-0, dim-1 int data type
    return df.shape[0], df.shape[1]

def check_nan(df): #return nan check result boolean data type
    nan_check = df.isnull().sum().tolist()
    flag = False
    for value in nan_check:
        if value !=0:
            flag = True
            break
    return flag
#https://gibles-deepmind.tistory.com/m/138 -> whether variable is numeric or categorical
def column_type_check(df):
    numeric_col = df._get_numeric_data().columns.tolist()
    categorical_col = list(set(df.columns) - set(numeric_col))

    return numeric_col, categorical_col


# Iris data

In [505]:
iris_df = make_df(iris)
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


## EDA

In [506]:
row_cnt, col_cnt = return_shape(iris_df)
nan_flag = check_nan(iris_df)
numeric_col, categorical_col = column_type_check(iris_df)

print("DataFrame shape : (" + str(row_cnt) + ", " + str(col_cnt)+")")
print("DataFrame has nan ? : " +str(nan_flag))
print("Numeric_col")
print(numeric_col)
print("Categorical_col")
print(categorical_col)

DataFrame shape : (150, 5)
DataFrame has nan ? : False
Numeric_col
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)', 'target']
Categorical_col
[]


## RandomForestClassifier

In [507]:
train_x, train_y = seperate_x_y(iris_df)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True, stratify=train_y)
print("Train size : " +str(train_x.shape))
print("Validation size: " +str(val_x.shape))


n_list = [i for i in range(10, 105, 5)]
best_n = 0
best_f1 = 0
for n in n_list:
    rf = RandomForestClassifier(n_estimators = n, criterion = 'entropy', random_state=SEED)
    rf.fit(train_x, train_y)
    pred = rf.predict(val_x)
    f1 = f1_score(val_y, pred, average='macro')
    if f1 > best_f1:
        best_n = n
        best_f1 = f1

rf_best = RandomForestClassifier(n_estimators = best_n, criterion = 'entropy', random_state=SEED)
rf_best.fit(train_x, train_y)
pred = rf_best.predict(val_x)

print("best_n : "+ str(best_n))
print("F1 score : " + str(f1_score(val_y, pred, average='macro')))
print("Accuracy score : " + str(accuracy_score(val_y, pred)))
print("confustion matrix")
print(confusion_matrix(val_y, pred))



Train size : (135, 4)
Validation size: (15, 4)
best_n : 10
F1 score : 0.9326599326599326
Accuracy score : 0.9333333333333333
confustion matrix
[[5 0 0]
 [0 4 1]
 [0 0 5]]


## Support Vector Machine

In [508]:
train_x, train_y = seperate_x_y(iris_df)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True, stratify=train_y)
print("Train size : " +str(train_x.shape))
print("Validation size: " +str(val_x.shape))

scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
val_x = scaler.transform(val_x)


c_list = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
best_c = 0
best_f1 = 0
for c in c_list:
    svm = SVC(C=c, gamma='auto', kernel='linear', random_state=SEED)
    svm.fit(train_x, train_y)
    pred = svm.predict(val_x)
    f1 = f1_score(val_y, pred, average='macro')
    if f1 > best_f1:
        best_c = c
        best_f1 = f1

svm_best =  SVC(C=c, gamma='auto', kernel='linear', random_state=SEED)
svm_best.fit(train_x, train_y)
pred = svm_best.predict(val_x)

print("best C : "+ str(best_c))
print("F1 score : " + str(f1_score(val_y, pred, average='macro')))
print("Accuracy score : " + str(accuracy_score(val_y, pred)))
print("confustion matrix")
print(confusion_matrix(val_y, pred))



Train size : (135, 4)
Validation size: (15, 4)
best C : 1000
F1 score : 1.0
Accuracy score : 1.0
confustion matrix
[[5 0 0]
 [0 5 0]
 [0 0 5]]


## Neural Network

In [509]:
class MyDataset(Dataset):
    def __init__(self, origin_x, origin_y):
        self.data = np.array(origin_x)
        self.labels = origin_y
    def __getitem__(self, index):
        self.x = self.data[index]
        self.y = self.labels[index]
        return torch.from_numpy(self.x), self.y

    def __len__(self):
        return len(self.data)

In [510]:
class Model(nn.Module):
    def __init__(self,input_size,output_size):
        super(Model, self).__init__()
        self.clf = nn.Sequential(
            nn.Linear(input_size,16),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(16,4),
            nn.ReLU(),
            nn.Linear(4,output_size),
        )
        self.weights_init()
        
    def forward(self, x):
        x = self.clf(x)
        return x
    
    def weights_init(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)

In [511]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        self.criterion = nn.CrossEntropyLoss().to(device)
    def fit(self, ):
        self.model.to(self.device)
        best_f1_score = 0
        best_loss = 1000000000
        best_true = []
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x, y in iter(self.train_loader):
                self.optimizer.zero_grad()
                x = x.float().to(self.device)
                y = y.to(self.device)
                y = torch.squeeze(y)
                _y = torch.squeeze(self.model(x)) 
                loss = self.criterion(_y, y)
                loss.backward()
                self.optimizer.step()
                train_loss.append(loss.item())
            mean_train_loss = np.mean(train_loss)
            
            f1_score ,acc, true, pred, val_loss = self.validation(self.model)
            print(f'Epoch : [{epoch+1}] Train loss : [{mean_train_loss}]\nVal f1 : [{f1_score}])\nVal acc :[{acc}]\nVal Loss : [{val_loss}]')
            print(confusion_matrix(true, pred))
            
            self.scheduler.step(val_loss)
                
            if best_loss > val_loss:
                best_f1_score = f1_score
                best_loss = val_loss
                torch.save(model.module.state_dict(), './model.pth', _use_new_zipfile_serialization=False)
                print("Best f1 changed : " + str(f1_score))
                print("Best val_loss changed : " + str(val_loss))
                print("Save Model ~_~")
                
        print(f"Best f1_Score : [{best_f1_score}]")
        print("Best val_loss : " + str(best_loss))
                
    
    def validation(self, eval_model):
        eval_model.eval()
        pred = []
        true = []
        val_loss = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)
                y = y.to(self.device)
                y = torch.squeeze(y)
                _y = torch.squeeze(eval_model(x))
                loss = self.criterion(_y,y)
                val_loss.append(loss.item())
                true += y.tolist()
                pred += torch.argmax(_y,dim=1).detach().tolist()
        return f1_score(true,pred,average='macro'), accuracy_score(true,pred),true, pred, np.mean(val_loss)
    
    

In [512]:
EPOCHS = 100
LR = 1e-3
BS = 8

device = torch.device('cpu') if torch.backends.mps.is_available() else torch.device('cpu')

#Seperate Data
train_x, train_y = seperate_x_y(iris_df)

#train, validation split
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True, stratify=train_y)

train_x.reset_index(drop=True,inplace=True)
train_y.reset_index(drop=True,inplace=True)
val_x.reset_index(drop=True,inplace=True)
val_y.reset_index(drop=True,inplace=True)
#Scaling
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
val_x = scaler.transform(val_x)

#Make custom dataset
train_dataset = MyDataset(train_x,train_y)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=0)
val_dataset = MyDataset(val_x,val_y)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=0)

input_size = train_x.shape[1]
output_size = 3

seed_everything(SEED) # Seed 고정 (모델 생성전에 다시 고정하였더니 재현이 똑같이됨..)

model = nn.DataParallel(Model(input_size, output_size))
optimizer = torch.optim.AdamW(params = model.parameters(), lr = LR, weight_decay=0.1)#L2 regularization
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, threshold_mode='abs', min_lr=1e-8, verbose=True)
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()


Epoch : [1] Train loss : [1.0769277144880856]
Val f1 : [0.2222222222222222])
Val acc :[0.3333333333333333]
Val Loss : [1.057235300540924]
[[0 5 0]
 [0 0 5]
 [0 0 5]]
Best f1 changed : 0.2222222222222222
Best val_loss changed : 1.057235300540924
Save Model ~_~
Epoch : [2] Train loss : [1.0797813219182633]
Val f1 : [0.2222222222222222])
Val acc :[0.3333333333333333]
Val Loss : [1.0397115349769592]
[[0 5 0]
 [0 0 5]
 [0 0 5]]
Best f1 changed : 0.2222222222222222
Best val_loss changed : 1.0397115349769592
Save Model ~_~
Epoch : [3] Train loss : [1.0728581091936897]
Val f1 : [0.2222222222222222])
Val acc :[0.3333333333333333]
Val Loss : [1.022508829832077]
[[0 5 0]
 [0 0 5]
 [0 0 5]]
Best f1 changed : 0.2222222222222222
Best val_loss changed : 1.022508829832077
Save Model ~_~
Epoch : [4] Train loss : [1.004443932982052]
Val f1 : [0.41269841269841273])
Val acc :[0.4666666666666667]
Val Loss : [1.0064403712749481]
[[2 3 0]
 [0 0 5]
 [0 0 5]]
Best f1 changed : 0.41269841269841273
Best val_loss

Epoch : [53] Train loss : [0.4852828032830182]
Val f1 : [0.6336996336996338])
Val acc :[0.6666666666666666]
Val Loss : [0.4532873034477234]
[[5 0 0]
 [0 1 4]
 [0 1 4]]
Best f1 changed : 0.6336996336996338
Best val_loss changed : 0.4532873034477234
Save Model ~_~
Epoch : [54] Train loss : [0.4652908733662437]
Val f1 : [0.6336996336996338])
Val acc :[0.6666666666666666]
Val Loss : [0.4504084140062332]
[[5 0 0]
 [0 1 4]
 [0 1 4]]
Best f1 changed : 0.6336996336996338
Best val_loss changed : 0.4504084140062332
Save Model ~_~
Epoch : [55] Train loss : [0.43462885565617504]
Val f1 : [0.7222222222222222])
Val acc :[0.7333333333333333]
Val Loss : [0.44749167561531067]
[[5 0 0]
 [0 2 3]
 [0 1 4]]
Best f1 changed : 0.7222222222222222
Best val_loss changed : 0.44749167561531067
Save Model ~_~
Epoch : [56] Train loss : [0.4670281725771287]
Val f1 : [0.6336996336996338])
Val acc :[0.6666666666666666]
Val Loss : [0.446647584438324]
[[5 0 0]
 [0 1 4]
 [0 1 4]]
Best f1 changed : 0.6336996336996338
Best

Epoch : [90] Train loss : [0.34908031102489023]
Val f1 : [0.8666666666666668])
Val acc :[0.8666666666666667]
Val Loss : [0.3795832097530365]
[[5 0 0]
 [0 4 1]
 [0 1 4]]
Best f1 changed : 0.8666666666666668
Best val_loss changed : 0.3795832097530365
Save Model ~_~
Epoch : [91] Train loss : [0.3124673778519911]
Val f1 : [0.8666666666666668])
Val acc :[0.8666666666666667]
Val Loss : [0.3774290680885315]
[[5 0 0]
 [0 4 1]
 [0 1 4]]
Best f1 changed : 0.8666666666666668
Best val_loss changed : 0.3774290680885315
Save Model ~_~
Epoch : [92] Train loss : [0.36393776886603413]
Val f1 : [0.8666666666666668])
Val acc :[0.8666666666666667]
Val Loss : [0.3762822300195694]
[[5 0 0]
 [0 4 1]
 [0 1 4]]
Best f1 changed : 0.8666666666666668
Best val_loss changed : 0.3762822300195694
Save Model ~_~
Epoch : [93] Train loss : [0.3295041050981073]
Val f1 : [0.8666666666666668])
Val acc :[0.8666666666666667]
Val Loss : [0.37411579489707947]
[[5 0 0]
 [0 4 1]
 [0 1 4]]
Best f1 changed : 0.8666666666666668
Bes

# diabetes data

In [513]:
diabetes_df = make_df(diabetes)
diabetes_df

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485,104.0
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491,132.0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930,220.0


## RandomForestRegressor

In [514]:
train_x, train_y = seperate_x_y(diabetes_df)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True)
print("Train size : " +str(train_x.shape))
print("Validation size: " +str(val_x.shape))


n_list = [i for i in range(10, 105, 5)]
best_n = 0
best_loss = 1000000000
for n in n_list:
    rf = RandomForestRegressor(n_estimators = n, random_state=SEED)
    rf.fit(train_x, train_y)
    pred = rf.predict(val_x)
    loss = mean_squared_error(val_y, pred)
    if loss < best_loss:
        best_n = n
        best_loss = loss
        
rf_best = RandomForestRegressor(n_estimators = best_n, random_state=SEED)
rf_best.fit(train_x, train_y)
pred = rf_best.predict(val_x)

print("best_n : "+ str(best_n))
print("MSE : " + str(mean_squared_error(val_y, pred)))



Train size : (397, 10)
Validation size: (45, 10)
best_n : 85
MSE : 2859.1126274509807


## Support Vector Machine

In [515]:
train_x, train_y = seperate_x_y(diabetes_df)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True)
print("Train size : " +str(train_x.shape))
print("Validation size: " +str(val_x.shape))

scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
val_x = scaler.transform(val_x)

c_list = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
best_c = 0
best_loss = 1000000000
for c in c_list:
    svm = SVR(C=c, gamma='auto')
    svm.fit(train_x, train_y)
    pred = svm.predict(val_x)
    loss = mean_squared_error(val_y, pred)
    if loss < best_loss:
        best_c = c
        best_loss = loss

svm_best = SVR(C=c, gamma='auto')
svm_best.fit(train_x, train_y)
pred = svm_best.predict(val_x)

print("best_c : "+ str(best_c))
print("MSE : " + str(mean_squared_error(val_y, pred)))


Train size : (397, 10)
Validation size: (45, 10)
best_c : 1000
MSE : 2440.2456618651254


## Neural Network

In [516]:
class MyDataset(Dataset):
    def __init__(self, origin_x, origin_y):
        self.data = np.array(origin_x)
        self.labels = origin_y
    def __getitem__(self, index):
        self.x = self.data[index]
        self.y = self.labels[index]
        return torch.from_numpy(self.x), self.y

    def __len__(self):
        return len(self.data)

In [517]:
class Model(nn.Module):
    def __init__(self,input_size,output_size):
        super(Model, self).__init__()
        self.clf = nn.Sequential(
            nn.Linear(input_size,16),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(16,4),
            nn.ReLU(),
            nn.Linear(4,output_size),
        )
        self.weights_init()
        
    def forward(self, x):
        x = self.clf(x)
        return x
    
    def weights_init(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)

In [518]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        self.criterion = nn.MSELoss().to(device)
        
    def fit(self, ):
        self.model.to(self.device)
        best_loss = 1000000000
        best_mse = 10000000
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x, y in iter(self.train_loader):
                self.optimizer.zero_grad()
                x = x.float().to(self.device)
                y = y.float().to(self.device)
                y = torch.squeeze(y)
                _y = torch.squeeze(self.model(x)).float()
                loss = self.criterion(_y, y)
                loss.backward()
                self.optimizer.step()
                train_loss.append(loss.item())
            mean_train_loss = np.mean(train_loss)
            
            mse, val_loss = self.validation(self.model)
            print(f'Epoch : [{epoch+1}] Train loss : [{mean_train_loss}]\nVal Loss : [{val_loss}]\nMSE :{mse}')
            
            self.scheduler.step(val_loss)
                
            if best_loss > val_loss:
                best_loss = val_loss
                best_mse = mse
                torch.save(model.module.state_dict(), './model.pth', _use_new_zipfile_serialization=False)
                print("Best MSE changed : " + str(mse))
                print("Save Model ~_~")
                
        print(f"Best MSE : [{best_mse}]")
                
    
    def validation(self, eval_model):
        eval_model.eval()
        val_loss = []
        pred = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)
                y = y.float().to(self.device)
                y = torch.squeeze(y)
                _y = torch.squeeze(eval_model(x)).float()
                loss = self.criterion(_y,y)
                val_loss.append(loss.item())
                pred += _y.detach().tolist()
        return mean_squared_error(val_y, pred), np.mean(val_loss)
    
    

In [519]:
EPOCHS = 200
LR = 1e-3
BS = 8

device = torch.device('cpu') if torch.backends.mps.is_available() else torch.device('cpu')

#Seperate Data
train_x, train_y = seperate_x_y(diabetes_df)

#train, validation split
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True)

train_x.reset_index(drop=True,inplace=True)
train_y.reset_index(drop=True,inplace=True)
val_x.reset_index(drop=True,inplace=True)
val_y.reset_index(drop=True,inplace=True)
#Scaling
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
val_x = scaler.transform(val_x)

#Make custom dataset
train_dataset = MyDataset(train_x,train_y)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=0)
val_dataset = MyDataset(val_x,val_y)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=0)

input_size = train_x.shape[1]
output_size = 1

seed_everything(SEED) # Seed 고정 (모델 생성전에 다시 고정하였더니 재현이 똑같이됨..)

model = nn.DataParallel(Model(input_size, output_size))
optimizer = torch.optim.AdamW(params = model.parameters(), lr = LR, weight_decay=1e-2)#L2 regularization
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, threshold_mode='abs', min_lr=1e-8, verbose=True)
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()


Epoch : [1] Train loss : [28684.46080078125]
Val Loss : [27910.72265625]
MSE :28815.229527762847
Best MSE changed : 28815.229527762847
Save Model ~_~
Epoch : [2] Train loss : [28082.643515625]
Val Loss : [27181.399739583332]
MSE :28072.073113456077
Best MSE changed : 28072.073113456077
Save Model ~_~
Epoch : [3] Train loss : [27325.12544921875]
Val Loss : [26047.201171875]
MSE :26916.49271061241
Best MSE changed : 26916.49271061241
Save Model ~_~
Epoch : [4] Train loss : [25740.39435546875]
Val Loss : [24250.46826171875]
MSE :25085.570159877454
Best MSE changed : 25085.570159877454
Save Model ~_~
Epoch : [5] Train loss : [23779.253583984377]
Val Loss : [21695.398111979168]
MSE :22479.57576325247
Best MSE changed : 22479.57576325247
Save Model ~_~
Epoch : [6] Train loss : [20917.5908984375]
Val Loss : [18444.071533203125]
MSE :19158.382745002764
Best MSE changed : 19158.382745002764
Save Model ~_~
Epoch : [7] Train loss : [17818.938984375]
Val Loss : [14947.032145182291]
MSE :15577.2759

Epoch : [57] Train loss : [4439.664870605468]
Val Loss : [3598.7421061197915]
MSE :3669.4840909696377
Best MSE changed : 3669.4840909696377
Save Model ~_~
Epoch : [58] Train loss : [4898.148623046875]
Val Loss : [3589.4964192708335]
MSE :3662.9263526537056
Best MSE changed : 3662.9263526537056
Save Model ~_~
Epoch : [59] Train loss : [4405.12544921875]
Val Loss : [3567.0814615885415]
MSE :3633.8074078883033
Best MSE changed : 3633.8074078883033
Save Model ~_~
Epoch : [60] Train loss : [4237.746618652343]
Val Loss : [3548.0030517578125]
MSE :3612.10778083407
Best MSE changed : 3612.10778083407
Save Model ~_~
Epoch : [61] Train loss : [4224.42460571289]
Val Loss : [3532.8201904296875]
MSE :3592.9957645288528
Best MSE changed : 3592.9957645288528
Save Model ~_~
Epoch : [62] Train loss : [4271.566440429688]
Val Loss : [3518.8521728515625]
MSE :3581.7342048876662
Best MSE changed : 3581.7342048876662
Save Model ~_~
Epoch : [63] Train loss : [4506.641673583985]
Val Loss : [3502.843058268229]

Epoch : [121] Train loss : [3888.796591796875]
Val Loss : [2988.005574544271]
MSE :3055.461531503919
Epoch : [122] Train loss : [3816.00833984375]
Val Loss : [2972.8045654296875]
MSE :3035.8617649330045
Epoch : [123] Train loss : [3698.4180432128906]
Val Loss : [2972.822021484375]
MSE :3038.7224248758243
Epoch : [124] Train loss : [4259.9996728515625]
Val Loss : [2982.6508585611978]
MSE :3054.654274598076
Epoch : [125] Train loss : [4222.117194824219]
Val Loss : [2977.1635538736978]
MSE :3048.941354376757
Epoch : [126] Train loss : [3984.682197265625]
Val Loss : [2976.3983357747397]
MSE :3049.5273150145363
Epoch 00126: reducing learning rate of group 0 to 5.0000e-04.
Epoch : [127] Train loss : [3682.5511926269533]
Val Loss : [2952.9515787760415]
MSE :3019.9772091874647
Best MSE changed : 3019.9772091874647
Save Model ~_~
Epoch : [128] Train loss : [3869.3189672851563]
Val Loss : [2947.376912434896]
MSE :3013.7432831318324
Best MSE changed : 3013.7432831318324
Save Model ~_~
Epoch : [12

Epoch : [192] Train loss : [3667.5959130859374]
Val Loss : [2913.0482177734375]
MSE :2982.554942249763
Epoch : [193] Train loss : [3770.497710571289]
Val Loss : [2912.8063354492188]
MSE :2982.259058852525
Epoch 00193: reducing learning rate of group 0 to 3.9063e-06.
Epoch : [194] Train loss : [3721.3134448242185]
Val Loss : [2912.8619995117188]
MSE :2982.328224734072
Epoch : [195] Train loss : [3669.7432495117187]
Val Loss : [2912.7347819010415]
MSE :2982.169396593816
Epoch : [196] Train loss : [3705.4575122070314]
Val Loss : [2912.7005004882812]
MSE :2982.132220004394
Epoch : [197] Train loss : [3759.151729736328]
Val Loss : [2912.720743815104]
MSE :2982.1633250926943
Epoch : [198] Train loss : [4070.618498535156]
Val Loss : [2912.5237833658853]
MSE :2981.9136125094037
Epoch : [199] Train loss : [3691.7880859375]
Val Loss : [2912.5066731770835]
MSE :2981.8996157766137
Epoch 00199: reducing learning rate of group 0 to 1.9531e-06.
Epoch : [200] Train loss : [3786.564617919922]
Val Loss 

# digits

In [520]:
digits_df = make_df(digits)
digits_df

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792,0.0,0.0,4.0,10.0,13.0,6.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,2.0,14.0,15.0,9.0,0.0,0.0,9
1793,0.0,0.0,6.0,16.0,13.0,11.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,16.0,14.0,6.0,0.0,0.0,0
1794,0.0,0.0,1.0,11.0,15.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,9.0,13.0,6.0,0.0,0.0,8
1795,0.0,0.0,2.0,10.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,12.0,16.0,12.0,0.0,0.0,9


## RandomForestClassifier

In [521]:
train_x, train_y = seperate_x_y(digits_df)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True, stratify=train_y)
print("Train size : " +str(train_x.shape))
print("Validation size: " +str(val_x.shape))


n_list = [i for i in range(10, 105, 5)]
best_n = 0
best_f1 = 0
for n in n_list:
    rf = RandomForestClassifier(n_estimators = n, criterion = 'entropy', random_state=SEED)
    rf.fit(train_x, train_y)
    pred = rf.predict(val_x)
    f1 = f1_score(val_y, pred, average='macro')
    if f1 > best_f1:
        best_n = n
        best_f1 = f1

rf_best = RandomForestClassifier(n_estimators = best_n, criterion = 'entropy', random_state=SEED)
rf_best.fit(train_x, train_y)
pred = rf_best.predict(val_x)

print("best_n : "+ str(best_n))
print("F1 score : " + str(f1_score(val_y, pred, average='macro')))
print("Accuracy score : " + str(accuracy_score(val_y, pred)))
print("confustion matrix")
print(confusion_matrix(val_y, pred))



Train size : (1617, 64)
Validation size: (180, 64)
best_n : 20
F1 score : 0.9780083509495274
Accuracy score : 0.9777777777777777
confustion matrix
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  0 18  0  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  0  0]
 [ 0  1  0  0 17  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  0  0  0 18  0  0  0]
 [ 0  0  0  0  0  0  0 18  0  0]
 [ 0  2  0  0  0  0  0  0 16  0]
 [ 0  0  0  1  0  0  0  0  0 17]]


## Support Vector Machine

In [522]:
train_x, train_y = seperate_x_y(digits_df)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True, stratify=train_y)
print("Train size : " +str(train_x.shape))
print("Validation size: " +str(val_x.shape))

scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
val_x = scaler.transform(val_x)

c_list = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
best_c = 0
best_f1 = 0
kernel = 'linear'
for c in c_list:
    svm = SVC(C=c, gamma='auto', kernel=kernel, random_state=SEED)
    svm.fit(train_x, train_y)
    pred = svm.predict(val_x)
    f1 = f1_score(val_y, pred, average='macro')
    if f1 > best_f1:
        best_c = c
        best_f1 = f1

svm_best =  SVC(C=c, gamma='auto',kernel=kernel, random_state=SEED)
svm_best.fit(train_x, train_y)
pred = svm_best.predict(val_x)

print("best C : "+ str(best_c))
print("F1 score : " + str(f1_score(val_y, pred, average='macro')))
print("Accuracy score : " + str(accuracy_score(val_y, pred)))
print("confustion matrix")
print(confusion_matrix(val_y, pred))



Train size : (1617, 64)
Validation size: (180, 64)
best C : 0.1
F1 score : 0.9832946436042412
Accuracy score : 0.9833333333333333
confustion matrix
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  0 18  0  0  0  0  0  0  0]
 [ 0  0  0 17  0  0  0  0  0  1]
 [ 0  0  0  0 18  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  0  0  0 18  0  0  0]
 [ 0  0  0  0  0  0  0 18  0  0]
 [ 0  2  0  0  0  0  0  0 16  0]
 [ 0  0  0  0  0  0  0  0  0 18]]


## Neural Network

In [523]:
class MyDataset(Dataset):
    def __init__(self, origin_x, origin_y):
        self.data = np.array(origin_x)
        self.labels = origin_y
    def __getitem__(self, index):
        self.x = self.data[index]
        self.y = self.labels[index]
        return torch.from_numpy(self.x), self.y

    def __len__(self):
        return len(self.data)

In [524]:
class Model(nn.Module):
    def __init__(self,input_size,output_size):
        super(Model, self).__init__()
        self.clf = nn.Sequential(
            nn.Linear(input_size,16),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(16,4),
            nn.ReLU(),
            nn.Linear(4,output_size),
        )
        self.weights_init()
        
    def forward(self, x):
        x = self.clf(x)
        return x
    
    def weights_init(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)

In [525]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        self.criterion = nn.CrossEntropyLoss().to(device)
    def fit(self, ):
        self.model.to(self.device)
        best_f1_score = 0
        best_loss = 1000000000
        best_true = []
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x, y in iter(self.train_loader):
                self.optimizer.zero_grad()
                x = x.float().to(self.device)
                y = y.to(self.device)
                y = torch.squeeze(y)
                _y = torch.squeeze(self.model(x)) 
                loss = self.criterion(_y, y)
                loss.backward()
                self.optimizer.step()
                train_loss.append(loss.item())
            mean_train_loss = np.mean(train_loss)
            
            f1_score ,acc, true, pred, val_loss = self.validation(self.model)
            print(f'Epoch : [{epoch+1}] Train loss : [{mean_train_loss}]\nVal f1 : [{f1_score}])\nVal acc :[{acc}]\nVal Loss : [{val_loss}]')
            print(confusion_matrix(true, pred))
            
            self.scheduler.step(val_loss)

            if best_loss > val_loss:
                best_f1_score = f1_score
                best_loss = val_loss
                torch.save(model.module.state_dict(), './model.pth', _use_new_zipfile_serialization=False)
                print("Best f1 changed : " + str(f1_score))
                print("Best val_loss changed : " + str(val_loss))
                print("Save Model ~_~")
                
        print(f"Best f1_Score : [{best_f1_score}]")
        print("Best val_loss : " + str(best_loss))
                
    
    def validation(self, eval_model):
        eval_model.eval()
        pred = []
        true = []
        val_loss = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)
                y = y.to(self.device)
                y = torch.squeeze(y)
                _y = torch.squeeze(eval_model(x))
                loss = self.criterion(_y,y)
                val_loss.append(loss.item())
                true += y.tolist()
                pred += torch.argmax(_y,dim=1).detach().tolist()
        return f1_score(true,pred,average='macro'), accuracy_score(true,pred),true, pred, np.mean(val_loss)
    
    

In [526]:
EPOCHS = 100
LR = 1e-3
BS = 8

device = torch.device('cpu') if torch.backends.mps.is_available() else torch.device('cpu')

#Seperate Data
train_x, train_y = seperate_x_y(digits_df)

#train, validation split
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True, stratify=train_y)

train_x.reset_index(drop=True,inplace=True)
train_y.reset_index(drop=True,inplace=True)
val_x.reset_index(drop=True,inplace=True)
val_y.reset_index(drop=True,inplace=True)
#Scaling
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
val_x = scaler.transform(val_x)

#Make custom dataset
train_dataset = MyDataset(train_x,train_y)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=0)
val_dataset = MyDataset(val_x,val_y)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=0)

input_size = train_x.shape[1]
output_size = 10

seed_everything(SEED) # Seed 고정 (모델 생성전에 다시 고정하였더니 재현이 똑같이됨..)

model = nn.DataParallel(Model(input_size, output_size))
optimizer = torch.optim.AdamW(params = model.parameters(), lr = LR, weight_decay=0.1)#L2 regularization
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, threshold_mode='abs', min_lr=1e-8, verbose=True)
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()


Epoch : [1] Train loss : [2.2242321879992932]
Val f1 : [0.13489352568951646])
Val acc :[0.25555555555555554]
Val Loss : [2.047145345936651]
[[ 0  0  0  0  0  0  8 10  0  0]
 [ 0  0  0  0  1 14  1  2  0  0]
 [ 0  0  0  0  0 16  0  2  0  0]
 [ 0  0  0  0  0 11  1  6  0  0]
 [ 2  0  0  0  0  1  9  4  2  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 1  0  0  0  2  1 11  1  2  0]
 [ 0  0  0  0  0  2  0 16  0  0]
 [ 1  0  0  0  0 12  0  4  1  0]
 [ 0  0  0  0  1  9  0  8  0  0]]
Best f1 changed : 0.13489352568951646
Best val_loss changed : 2.047145345936651
Save Model ~_~
Epoch : [2] Train loss : [2.0428345450039567]
Val f1 : [0.2608709637143397])
Val acc :[0.3611111111111111]
Val Loss : [1.8872668743133545]
[[ 3  0  0  0  0  0 10  4  1  0]
 [ 0  0  0  1  2  5  3  3  4  0]
 [ 0  0  0  0  0 16  0  1  1  0]
 [ 0  0  0  0  1  7  2  8  0  0]
 [ 1  0  0  0 10  0  5  2  0  0]
 [ 0  0  0  0  1 17  0  0  0  0]
 [ 0  0  0  0  5  0  9  0  4  0]
 [ 0  0  0  0  0  0  0 18  0  0]
 [ 0  0  0  0  0  4  1  5  8  0

Epoch : [17] Train loss : [0.7929603481674429]
Val f1 : [0.8388484778298022])
Val acc :[0.8444444444444444]
Val Loss : [0.6048984436885171]
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  1  1  2  0  0  2]
 [ 0  0 16  0  0  0  0  0  2  0]
 [ 0  0  1 14  0  0  0  0  0  3]
 [ 0  0  0  0 18  0  0  0  0  0]
 [ 0  0  0  1  0 17  0  0  0  0]
 [ 0  0  0  0  0  0 18  0  0  0]
 [ 0  0  0  0  0  0  0 18  0  0]
 [ 0  5  0  0  0  0  0  0 13  0]
 [ 0  2  0  4  0  0  0  4  0  8]]
Best f1 changed : 0.8388484778298022
Best val_loss changed : 0.6048984436885171
Save Model ~_~
Epoch : [18] Train loss : [0.8105217974467818]
Val f1 : [0.8409650611112459])
Val acc :[0.8444444444444444]
Val Loss : [0.587834885586863]
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 13  0  0  1  0  2  0  0  2]
 [ 0  0 17  0  0  0  0  0  1  0]
 [ 0  0  1 14  0  0  0  0  0  3]
 [ 0  0  0  0 18  0  0  0  0  0]
 [ 0  0  0  2  0 16  0  0  0  0]
 [ 0  1  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  0  0  0 18  0  0]
 [ 0  5  0  0  0  0  0  0 13  0

Epoch : [33] Train loss : [0.5815528961898658]
Val f1 : [0.8872223523488586])
Val acc :[0.8888888888888888]
Val Loss : [0.39275433317474695]
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  2  0  0  2]
 [ 0  0 17  0  0  0  0  0  1  0]
 [ 0  0  0 18  0  0  0  0  0  0]
 [ 0  0  0  0 18  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  1  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  1  0  0 17  0  0]
 [ 0  6  0  0  0  0  0  0 12  0]
 [ 0  2  0  2  0  0  0  3  0 11]]
Best f1 changed : 0.8872223523488586
Best val_loss changed : 0.39275433317474695
Save Model ~_~
Epoch : [34] Train loss : [0.6015584137052151]
Val f1 : [0.9049847285615785])
Val acc :[0.9055555555555556]
Val Loss : [0.3886163694703061]
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 15  0  0  0  0  1  0  0  2]
 [ 0  0 18  0  0  0  0  0  0  0]
 [ 0  0  0 17  0  0  0  0  0  1]
 [ 0  0  0  0 18  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  1  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  1  0  0 17  0  0]
 [ 0  4  0  0  0  0  0  0 13

Epoch : [49] Train loss : [0.5112415874661337]
Val f1 : [0.9166143860880702])
Val acc :[0.9166666666666666]
Val Loss : [0.32528291577878204]
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  2  0  1  1]
 [ 0  0 18  0  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  0  0]
 [ 0  0  0  0 16  0  1  1  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  1  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  1  0  0 17  0  0]
 [ 0  3  0  0  0  0  0  0 14  1]
 [ 0  1  0  0  0  0  0  2  0 15]]
Best f1 changed : 0.9166143860880702
Best val_loss changed : 0.32528291577878204
Save Model ~_~
Epoch : [50] Train loss : [0.5138074288172234]
Val f1 : [0.8995042738961224])
Val acc :[0.9]
Val Loss : [0.31699333566686383]
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 15  0  0  0  0  1  0  1  1]
 [ 0  0 18  0  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  0  0]
 [ 0  1  0  0 16  0  0  1  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  1  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  1  0  0 17  0  0]
 [ 0  4  0  0  0  0  0  0 13  1]
 [ 0  2  

Epoch : [67] Train loss : [0.4414688561643873]
Val f1 : [0.9219654407154408])
Val acc :[0.9222222222222223]
Val Loss : [0.2923653951805571]
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 15  0  0  0  0  1  0  1  1]
 [ 0  0 18  0  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  0  0]
 [ 0  0  0  0 17  0  0  1  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  1  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  1  0  0 17  0  0]
 [ 0  4  0  0  0  0  0  0 13  1]
 [ 0  1  0  1  0  0  0  1  0 15]]
Epoch : [68] Train loss : [0.46100491670698956]
Val f1 : [0.9336180823680824])
Val acc :[0.9333333333333333]
Val Loss : [0.28707716225282004]
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  0  0  1  1]
 [ 0  0 18  0  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  0  0]
 [ 0  0  0  0 17  0  0  1  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  1  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  1  0  0 17  0  0]
 [ 0  4  0  0  0  0  0  0 13  1]
 [ 0  1  0  0  0  0  0  1  0 16]]
Best f1 changed : 0.9336180823680824
Best val_loss chang

Epoch : [84] Train loss : [0.4481276519266255]
Val f1 : [0.9218136739698709])
Val acc :[0.9222222222222223]
Val Loss : [0.2855361434428588]
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  0  0  1  1]
 [ 0  0 18  0  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  0  0]
 [ 0  0  0  0 17  0  0  1  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  1  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  1  0  0 17  0  0]
 [ 0  4  0  0  0  0  0  0 12  2]
 [ 0  1  0  0  0  0  0  2  0 15]]
Epoch : [85] Train loss : [0.4175761551716654]
Val f1 : [0.9273850348043895])
Val acc :[0.9277777777777778]
Val Loss : [0.2870977512844231]
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  0  0  1  1]
 [ 0  0 18  0  0  0  0  0  0  0]
 [ 0  0  0 18  0  0  0  0  0  0]
 [ 0  0  0  0 17  0  0  1  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  1  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  1  0  0 17  0  0]
 [ 0  4  0  0  0  0  0  0 12  2]
 [ 0  1  0  0  0  0  0  1  0 16]]
Epoch : [86] Train loss : [0.41377036640503134]
Val f1 : [

# wine

In [527]:
wine_df = make_df(wine)
wine_df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


## RandomForestClassifier

In [528]:
train_x, train_y = seperate_x_y(wine_df)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True, stratify=train_y)
print("Train size : " +str(train_x.shape))
print("Validation size: " +str(val_x.shape))


n_list = [i for i in range(10, 105, 5)]
best_n = 0
best_f1 = 0
for n in n_list:
    rf = RandomForestClassifier(n_estimators = n, criterion = 'entropy', random_state=SEED)
    rf.fit(train_x, train_y)
    pred = rf.predict(val_x)
    f1 = f1_score(val_y, pred, average='macro')
    if f1 > best_f1:
        best_n = n
        best_f1 = f1

rf_best = RandomForestClassifier(n_estimators = best_n, criterion = 'entropy', random_state=SEED)
rf_best.fit(train_x, train_y)
pred = rf_best.predict(val_x)

print("best_n : "+ str(best_n))
print("F1 score : " + str(f1_score(val_y, pred, average='macro')))
print("Accuracy score : " + str(accuracy_score(val_y, pred)))
print("confustion matrix")
print(confusion_matrix(val_y, pred))



Train size : (160, 13)
Validation size: (18, 13)
best_n : 10
F1 score : 1.0
Accuracy score : 1.0
confustion matrix
[[6 0 0]
 [0 7 0]
 [0 0 5]]


## Support Vector Machine

In [529]:
train_x, train_y = seperate_x_y(wine_df)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True, stratify=train_y)
print("Train size : " +str(train_x.shape))
print("Validation size: " +str(val_x.shape))

scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
val_x = scaler.transform(val_x)

c_list = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
kernel = 'linear'
best_c = 0
best_f1 = 0
for c in c_list:
    svm = SVC(C=c, gamma='auto', kernel=kernel, random_state=SEED)
    svm.fit(train_x, train_y)
    pred = svm.predict(val_x)
    f1 = f1_score(val_y, pred, average='macro')
    if f1 > best_f1:
        best_c = c
        best_f1 = f1

svm_best = SVC(C=c, gamma='auto', kernel=kernel, random_state=SEED)
svm_best.fit(train_x, train_y)
pred = svm_best.predict(val_x)

print("best C : "+ str(best_c))
print("F1 score : " + str(f1_score(val_y, pred, average='macro')))
print("Accuracy score : " + str(accuracy_score(val_y, pred)))
print("confustion matrix")
print(confusion_matrix(val_y, pred))



Train size : (160, 13)
Validation size: (18, 13)
best C : 0.1
F1 score : 1.0
Accuracy score : 1.0
confustion matrix
[[6 0 0]
 [0 7 0]
 [0 0 5]]


## Neural Network

In [530]:
class MyDataset(Dataset):
    def __init__(self, origin_x, origin_y):
        self.data = np.array(origin_x)
        self.labels = origin_y
    def __getitem__(self, index):
        self.x = self.data[index]
        self.y = self.labels[index]
        return torch.from_numpy(self.x), self.y

    def __len__(self):
        return len(self.data)

In [531]:
class Model(nn.Module):
    def __init__(self,input_size,output_size):
        super(Model, self).__init__()
        self.clf = nn.Sequential(
            nn.Linear(input_size,16),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(16,4),
            nn.ReLU(),
            nn.Linear(4,output_size),
        )
        self.weights_init()
        
    def forward(self, x):
        x = self.clf(x)
        return x
    
    def weights_init(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)

In [532]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        self.criterion = nn.CrossEntropyLoss().to(device)
    def fit(self, ):
        self.model.to(self.device)
        best_f1_score = 0
        best_loss = 1000000000
        best_true = []
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x, y in iter(self.train_loader):
                self.optimizer.zero_grad()
                x = x.float().to(self.device)
                y = y.to(self.device)
                y = torch.squeeze(y)
                _y = torch.squeeze(self.model(x)) 
                loss = self.criterion(_y, y)
                loss.backward()
                self.optimizer.step()
                train_loss.append(loss.item())
            mean_train_loss = np.mean(train_loss)
            
            f1_score ,acc, true, pred, val_loss = self.validation(self.model)
            print(f'Epoch : [{epoch+1}] Train loss : [{mean_train_loss}]\nVal f1 : [{f1_score}])\nVal acc :[{acc}]\nVal Loss : [{val_loss}]')
            print(confusion_matrix(true, pred))
            
            self.scheduler.step(val_loss)
                

            if best_loss > val_loss:
                best_f1_score = f1_score
                best_loss = val_loss
                torch.save(model.module.state_dict(), './model.pth', _use_new_zipfile_serialization=False)
                print("Best f1 changed : " + str(f1_score))
                print("Best val_loss changed : " + str(val_loss))
                print("Save Model ~_~")
                
        print(f"Best f1_Score : [{best_f1_score}]")
        print("Best val_loss : " + str(best_loss))
                
    
    def validation(self, eval_model):
        eval_model.eval()
        pred = []
        true = []
        val_loss = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)
                y = y.to(self.device)
                y = torch.squeeze(y)
                _y = torch.squeeze(eval_model(x))
                loss = self.criterion(_y,y)
                val_loss.append(loss.item())
                true += y.tolist()
                pred += torch.argmax(_y,dim=1).detach().tolist()
        return f1_score(true,pred,average='macro'), accuracy_score(true,pred),true, pred, np.mean(val_loss)
    
    

In [533]:
EPOCHS = 100
LR = 1e-3
BS = 8

device = torch.device('cpu') if torch.backends.mps.is_available() else torch.device('cpu')

#Seperate Data
train_x, train_y = seperate_x_y(wine_df)

#train, validation split
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True, stratify=train_y)

train_x.reset_index(drop=True,inplace=True)
train_y.reset_index(drop=True,inplace=True)
val_x.reset_index(drop=True,inplace=True)
val_y.reset_index(drop=True,inplace=True)
#Scaling
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
val_x = scaler.transform(val_x)

#Make custom dataset
train_dataset = MyDataset(train_x,train_y)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=0)
val_dataset = MyDataset(val_x,val_y)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=0)

input_size = train_x.shape[1]
output_size = 3

seed_everything(SEED) # Seed 고정 (모델 생성전에 다시 고정하였더니 재현이 똑같이됨..)

model = nn.DataParallel(Model(input_size, output_size))
optimizer = torch.optim.AdamW(params = model.parameters(), lr = LR, weight_decay=0.1)#L2 regularization
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, threshold_mode='abs', min_lr=1e-8, verbose=True)
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()


Epoch : [1] Train loss : [1.1047492682933808]
Val f1 : [0.18666666666666668])
Val acc :[0.3888888888888889]
Val Loss : [1.0980699459711711]
[[0 6 0]
 [0 7 0]
 [0 5 0]]
Best f1 changed : 0.18666666666666668
Best val_loss changed : 1.0980699459711711
Save Model ~_~
Epoch : [2] Train loss : [1.1011233627796173]
Val f1 : [0.18666666666666668])
Val acc :[0.3888888888888889]
Val Loss : [1.0972199042638142]
[[0 6 0]
 [0 7 0]
 [0 5 0]]
Best f1 changed : 0.18666666666666668
Best val_loss changed : 1.0972199042638142
Save Model ~_~
Epoch : [3] Train loss : [1.0908971190452577]
Val f1 : [0.18666666666666668])
Val acc :[0.3888888888888889]
Val Loss : [1.0956267515818279]
[[0 6 0]
 [0 7 0]
 [0 5 0]]
Best f1 changed : 0.18666666666666668
Best val_loss changed : 1.0956267515818279
Save Model ~_~
Epoch : [4] Train loss : [1.0848861277103423]
Val f1 : [0.18666666666666668])
Val acc :[0.3888888888888889]
Val Loss : [1.0933170318603516]
[[0 6 0]
 [0 7 0]
 [0 5 0]]
Best f1 changed : 0.18666666666666668
Be

Epoch : [47] Train loss : [0.22619290165603162]
Val f1 : [1.0])
Val acc :[1.0]
Val Loss : [0.15178054322799048]
[[6 0 0]
 [0 7 0]
 [0 0 5]]
Epoch : [48] Train loss : [0.2874672245234251]
Val f1 : [1.0])
Val acc :[1.0]
Val Loss : [0.14969555288553238]
[[6 0 0]
 [0 7 0]
 [0 0 5]]
Epoch : [49] Train loss : [0.2628536753356457]
Val f1 : [1.0])
Val acc :[1.0]
Val Loss : [0.15949702262878418]
[[6 0 0]
 [0 7 0]
 [0 0 5]]
Epoch : [50] Train loss : [0.2049765232950449]
Val f1 : [1.0])
Val acc :[1.0]
Val Loss : [0.14562134072184563]
[[6 0 0]
 [0 7 0]
 [0 0 5]]
Best f1 changed : 1.0
Best val_loss changed : 0.14562134072184563
Save Model ~_~
Epoch : [51] Train loss : [0.2720337510108948]
Val f1 : [1.0])
Val acc :[1.0]
Val Loss : [0.132167379061381]
[[6 0 0]
 [0 7 0]
 [0 0 5]]
Best f1 changed : 1.0
Best val_loss changed : 0.132167379061381
Save Model ~_~
Epoch : [52] Train loss : [0.2721910173073411]
Val f1 : [0.9487179487179486])
Val acc :[0.9444444444444444]
Val Loss : [0.1275607024629911]
[[6 0 

# cancer

In [534]:
cancer_df = make_df(cancer)
cancer_df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


## RandomForestClassifier

In [535]:
train_x, train_y = seperate_x_y(cancer_df)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True, stratify=train_y)
print("Train size : " +str(train_x.shape))
print("Validation size: " +str(val_x.shape))


n_list = [i for i in range(10, 105, 5)]
best_n = 0
best_f1 = 0
for n in n_list:
    rf = RandomForestClassifier(n_estimators = n, criterion = 'entropy', random_state=SEED)
    rf.fit(train_x, train_y)
    pred = rf.predict(val_x)
    f1 = f1_score(val_y, pred, average='macro')
    if f1 > best_f1:
        best_n = n
        best_f1 = f1

rf_best = RandomForestClassifier(n_estimators = best_n, criterion = 'entropy', random_state=SEED)
rf_best.fit(train_x, train_y)
pred = rf.predict(val_x)

print("best_n : "+ str(best_n))
print("F1 score : " + str(f1_score(val_y, pred, average='macro')))
print("Accuracy score : " + str(accuracy_score(val_y, pred)))
print("confustion matrix")
print(confusion_matrix(val_y, pred))



Train size : (512, 30)
Validation size: (57, 30)
best_n : 15
F1 score : 0.9623015873015872
Accuracy score : 0.9649122807017544
confustion matrix
[[20  1]
 [ 1 35]]


## Support Vector Machine

In [536]:
train_x, train_y = seperate_x_y(cancer_df)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True, stratify=train_y)
print("Train size : " +str(train_x.shape))
print("Validation size: " +str(val_x.shape))

scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
val_x = scaler.transform(val_x)

c_list = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
best_c = 0
best_f1 = 0
for c in c_list:
    svm = SVC(C=c, gamma='auto', random_state=SEED)
    svm.fit(train_x, train_y)
    pred = svm.predict(val_x)
    f1 = f1_score(val_y, pred, average='macro')
    if f1 > best_f1:
        best_c = c
        best_f1 = f1

svm_best = SVC(C=c, gamma='auto', random_state=SEED)
svm_best.fit(train_x, train_y)
pred = svm_best.predict(val_x)

print("best C : "+ str(best_c))
print("F1 score : " + str(f1_score(val_y, pred, average='macro')))
print("Accuracy score : " + str(accuracy_score(val_y, pred)))
print("confustion matrix")
print(confusion_matrix(val_y, pred))



Train size : (512, 30)
Validation size: (57, 30)
best C : 1
F1 score : 0.9439895185063871
Accuracy score : 0.9473684210526315
confustion matrix
[[20  1]
 [ 2 34]]


## Neural Network

In [537]:
class MyDataset(Dataset):
    def __init__(self, origin_x, origin_y):
        self.data = np.array(origin_x)
        self.labels = origin_y
    def __getitem__(self, index):
        self.x = self.data[index]
        self.y = self.labels[index]
        return torch.from_numpy(self.x), self.y

    def __len__(self):
        return len(self.data)

In [538]:
class Model(nn.Module):
    def __init__(self,input_size,output_size):
        super(Model, self).__init__()
        self.clf = nn.Sequential(
            nn.Linear(input_size,16),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(16,4),
            nn.ReLU(),
            nn.Linear(4,output_size),
        )
        self.weights_init()
        
    def forward(self, x):
        x = self.clf(x)
        return x
    
    def weights_init(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)

In [539]:
class Trainer():
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        self.criterion = nn.BCEWithLogitsLoss().to(device)
    def fit(self, ):
        self.model.to(self.device)
        best_f1_score = 0
        best_loss = 1000000000
        best_true = []
        for epoch in range(EPOCHS):
            self.model.train()
            train_loss = []
            for x, y in iter(self.train_loader):
                self.optimizer.zero_grad()
                x = x.float().to(self.device)
                y = y.float().to(self.device)
                y = torch.squeeze(y)
                _y = torch.squeeze(self.model(x)).float()
                loss = self.criterion(_y, y)
                loss.backward()
                self.optimizer.step()
                train_loss.append(loss.item())
            mean_train_loss = np.mean(train_loss)
            
            f1_score ,acc, true, pred, val_loss = self.validation(self.model)
            print(f'Epoch : [{epoch+1}] Train loss : [{mean_train_loss}]\nVal f1 : [{f1_score}])\nVal acc :[{acc}]\nVal Loss : [{val_loss}]')
            print(confusion_matrix(true, pred))
            
            self.scheduler.step(val_loss)
            
            if best_loss > val_loss:
                best_f1_score = f1_score
                best_loss = val_loss
                torch.save(model.module.state_dict(), './model.pth', _use_new_zipfile_serialization=False)
                print("Best f1 changed : " + str(f1_score))
                print("Best val_loss changed : " + str(val_loss))
                print("Save Model ~_~")
                
        print(f"Best f1_Score : [{best_f1_score}]")
        print("Best val_loss : " + str(best_loss))
                
    
    def validation(self, eval_model):
        eval_model.eval()
        pred = []
        true = []
        val_loss = []
        with torch.no_grad():
            for x, y in iter(self.val_loader):
                x = x.float().to(self.device)
                y = y.float().to(self.device)
                y = torch.squeeze(y)
                _y = torch.squeeze(eval_model(x)).float()
                loss = self.criterion(_y,y)
                val_loss.append(loss.item())
                true.extend(y.tolist())
                pred.extend(torch.where(_y > 0,1,0).detach().tolist())
        return f1_score(true,pred,average='macro'), accuracy_score(true,pred),true, pred, np.mean(val_loss)
    
    

In [540]:
EPOCHS = 100
LR = 1e-3
BS = 16

device = torch.device('cpu') if torch.backends.mps.is_available() else torch.device('cpu')

#Seperate Data
train_x, train_y = seperate_x_y(cancer_df)

#train, validation split
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state=SEED, shuffle=True, stratify=train_y)

train_x.reset_index(drop=True,inplace=True)
train_y.reset_index(drop=True,inplace=True)
val_x.reset_index(drop=True,inplace=True)
val_y.reset_index(drop=True,inplace=True)

#Scaling
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
val_x = scaler.transform(val_x)

#Make custom dataset
train_dataset = MyDataset(train_x,train_y)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=0)
val_dataset = MyDataset(val_x,val_y)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False, num_workers=0)

input_size = train_x.shape[1]
output_size = 1

seed_everything(SEED) # Seed 고정 (모델 생성전에 다시 고정하였더니 재현이 똑같이됨..)

model = nn.DataParallel(Model(input_size, output_size))
optimizer = torch.optim.AdamW(params = model.parameters(), lr = LR, weight_decay=0.1)#L2 regularization
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, threshold_mode='abs', min_lr=1e-8, verbose=True)
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)
trainer.fit()


Epoch : [1] Train loss : [0.6826768051832914]
Val f1 : [0.3870967741935484])
Val acc :[0.631578947368421]
Val Loss : [0.6750159412622452]
[[ 0 21]
 [ 0 36]]
Best f1 changed : 0.3870967741935484
Best val_loss changed : 0.6750159412622452
Save Model ~_~
Epoch : [2] Train loss : [0.6704766564071178]
Val f1 : [0.3870967741935484])
Val acc :[0.631578947368421]
Val Loss : [0.6588680148124695]
[[ 0 21]
 [ 0 36]]
Best f1 changed : 0.3870967741935484
Best val_loss changed : 0.6588680148124695
Save Model ~_~
Epoch : [3] Train loss : [0.6446704156696796]
Val f1 : [0.3870967741935484])
Val acc :[0.631578947368421]
Val Loss : [0.6338419616222382]
[[ 0 21]
 [ 0 36]]
Best f1 changed : 0.3870967741935484
Best val_loss changed : 0.6338419616222382
Save Model ~_~
Epoch : [4] Train loss : [0.626183370128274]
Val f1 : [0.3870967741935484])
Val acc :[0.631578947368421]
Val Loss : [0.6119824945926666]
[[ 0 21]
 [ 0 36]]
Best f1 changed : 0.3870967741935484
Best val_loss changed : 0.6119824945926666
Save Mod

[[20  1]
 [ 0 36]]
Best f1 changed : 0.9809555629802873
Best val_loss changed : 0.20200694538652897
Save Model ~_~
Epoch : [48] Train loss : [0.18608752242289484]
Val f1 : [0.9623015873015872])
Val acc :[0.9649122807017544]
Val Loss : [0.20484615676105022]
[[20  1]
 [ 1 35]]
Epoch : [49] Train loss : [0.1891480062622577]
Val f1 : [0.9809555629802873])
Val acc :[0.9824561403508771]
Val Loss : [0.19700976461172104]
[[20  1]
 [ 0 36]]
Best f1 changed : 0.9809555629802873
Best val_loss changed : 0.19700976461172104
Save Model ~_~
Epoch : [50] Train loss : [0.1876287132035941]
Val f1 : [0.9809555629802873])
Val acc :[0.9824561403508771]
Val Loss : [0.18859137035906315]
[[20  1]
 [ 0 36]]
Best f1 changed : 0.9809555629802873
Best val_loss changed : 0.18859137035906315
Save Model ~_~
Epoch : [51] Train loss : [0.1902056448161602]
Val f1 : [0.9439895185063871])
Val acc :[0.9473684210526315]
Val Loss : [0.21433578990399837]
[[20  1]
 [ 2 34]]
Epoch : [52] Train loss : [0.17920736852101982]
Val 

Epoch : [93] Train loss : [0.12880828813649714]
Val f1 : [0.9623015873015872])
Val acc :[0.9649122807017544]
Val Loss : [0.1494423896074295]
[[20  1]
 [ 1 35]]
Best f1 changed : 0.9623015873015872
Best val_loss changed : 0.1494423896074295
Save Model ~_~
Epoch : [94] Train loss : [0.128971605328843]
Val f1 : [0.9623015873015872])
Val acc :[0.9649122807017544]
Val Loss : [0.15027851052582264]
[[20  1]
 [ 1 35]]
Epoch : [95] Train loss : [0.13149252336006612]
Val f1 : [0.9623015873015872])
Val acc :[0.9649122807017544]
Val Loss : [0.14994017034769058]
[[20  1]
 [ 1 35]]
Epoch : [96] Train loss : [0.1281080807093531]
Val f1 : [0.9623015873015872])
Val acc :[0.9649122807017544]
Val Loss : [0.1519217025488615]
[[20  1]
 [ 1 35]]
Epoch : [97] Train loss : [0.12617855099961162]
Val f1 : [0.9623015873015872])
Val acc :[0.9649122807017544]
Val Loss : [0.15330973453819752]
[[20  1]
 [ 1 35]]
Epoch : [98] Train loss : [0.12233431183267385]
Val f1 : [0.9623015873015872])
Val acc :[0.96491228070175